Speed Detection and Car tracking

In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

try :
    from tracker import *
except ImportError :
    print('Import not done')

In [2]:
import ultralytics
import cv2
import pandas as pd
from ultralytics import YOLO


In [3]:
model = YOLO('yolov8s.pt')

In [4]:
class_list = ['person', 'bicycle', 'car','motorcycle', 'airplane', 'bus',
              'train', 'truck', 'traffic light']

# class_list = ['car','truck']

In [5]:
tracker = Tracker()
count = 0

In [6]:
down = {}

In [7]:
cap = cv2.VideoCapture('C:\\Users\\Aman\\Desktop\\thesisproject\\video\\highway (1).mp4')

In [ ]:
# Open the video file
cap = cv2.VideoCapture('C:\\Users\\Aman\\Desktop\\thesisproject\\video\\highway (1).mp4')

# Get the frames per second (FPS) of the video
fps = cap.get(cv2.CAP_PROP_FPS)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    result = model.predict(frame)
    a = result[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        x2 = int(row[2])
        y1 = int(row[1])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        red_line_y = 198
        blue_line_y = 268
        offset = 7

        # Condition for red line
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
            down[id] = cy
            if id in down:
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
        

    text_color = (255, 255, 255)
    red_color = (0, 0, 255)
    blue_color = (255, 0, 0)
    green_color = (0, 255, 0)

    cv2.line(frame, (172, 198), (774, 198), red_color, 3)
    cv2.putText(frame, ('red line'), (172, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (8, 268), (927, 268), blue_color, 3)
    cv2.putText(frame, ('blue line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    # Show the frame
    cv2.imshow('frames', frame)

    # Wait for the appropriate time based on FPS (frame delay in milliseconds)
    delay = int(1000 / fps)  # Delay per frame in milliseconds
    if cv2.waitKey(delay) & 0xFF == 27:  # ESC key to exit
        break

cap.release()
cv2.destroyAllWindows()



0: 320x640 8 cars, 2 trucks, 2483.0ms
Speed: 381.6ms preprocess, 2483.0ms inference, 185.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 352.5ms
Speed: 9.3ms preprocess, 352.5ms inference, 29.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 2 trucks, 370.2ms
Speed: 2.0ms preprocess, 370.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 2 trucks, 474.0ms
Speed: 43.4ms preprocess, 474.0ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 413.9ms
Speed: 0.0ms preprocess, 413.9ms inference, 7.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 348.2ms
Speed: 11.5ms preprocess, 348.2ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 336.5ms
Speed: 4.4ms preprocess, 336.5ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 trucks, 368.7ms
S

IndexError: list index out of range

: 